plot_change.py: Generate significance of electrode weight changes at neuroanatomical sites

## Pseudocode:
    . Import
    . Define functions
    . Input symptom, mode, perdur
    . Load Parameters
    . For Loop through seizures

## Import

In [1]:
import csv
import os
import pathlib
import numpy as np
from scipy.io import loadmat
from scipy import signal

#
import pandas as pd
import datetime
# from plot_change_module import *

## Define Functions

In [2]:
def open_csv(mat_file):
    
    with open(mat_file,'r') as sz_mat:
        mat_csv_list = list(csv.reader(sz_mat))

        all_sx = mat_csv_list[0] #collect symptom abbreviations
        sx_col = all_sx.index(sx_input) #find col num of symptom
        csv_rows = np.size(mat_csv_list,0) #find row num 

        for r in range(1,csv_rows):  #collect symptom data for specfic seizure in list
            sx_data.append(mat_csv_list[r][sx_col]) 

        try: 
            mx_index = sx_data.index(mx_input) #find row num of first occurance of mode  
            return mx_index

        except ValueError:
            print('no matching mode')
            

In [3]:
def open_xl(xl_name): 
    # df = pd.read_excel(xl_name)
    # Ndimensions = df.shape #row, col
    # params_list = df.values.tolist()
    
    df = pd.read_excel(xl_name, index_col=0, engine='openpyxl')

    return df


In [4]:
def ll_weights(sz_name,perdur_input,em_row,params_llw,ppEEG,t_ppEEG,df,fs):
    try:
        mx_index = df.index[df[sx_input]==int(mx_input)] # time of symptom 
            #load fs, ppEEG, study

        ppEEG_col = len(sz_mat['ppEEG'][0])
        ppEEG_row = len(sz_mat['ppEEG'])

        try: #find index of symptom onset 
            ppEEG_mx = round(((mx_index[0])/5)*fs[0])
            print(ppEEG_mx)
            ppEEG_perdur = int(perdur_input) * fs[0]

            ppEEG_onset_list = []
            ppEEG_before_list = []
            ppEEG_after_list = []

                  
            for p in range(0,em_row): #create lists of symptom onset times before and after perdur
                ppEEG_onset_list.append(ppEEG[p][ppEEG_mx]) #at exact onset
                # t_ppEEG_onset_list.append(t_ppEEG[ppEEG_mx][p]) #at exact onset

                ppEEG_before_list.append(ppEEG[p][ppEEG_mx-ppEEG_perdur])  #perdur before
                ppEEG_after_list.append(ppEEG[p][ppEEG_mx+ppEEG_perdur])   #perdur after
            return ppEEG_onset_list,ppEEG_before_list,ppEEG_after_list#,t_ppEEG_onset_list
            # return t_ppEEG_onset_list
        except: 
            print('no ppEEG_mx')

    except IndexError:
        print('no index or ppEEG')


In [5]:
def load_elecs_anat(pt_path):
                
    #load anatomy and electrode matrix
    os.chdir(pt_path + 'Imaging/elecs')
    e_mat = loadmat('clinical_elecs_all.mat')
    anatomy = e_mat['anatomy']
    em = e_mat['elecmatrix']
    em_row = np.shape(anatomy)[0]
    
    return em_row, em, anatomy


In [6]:
def get_params(df_params,pt):
    params_bl_start = df_params.loc[pt]['BLstart']
    params_bl_stop = df_params.loc[pt]['BLstop']
    params_llw = df_params.loc[pt]['llw']

    return params_bl_start, params_bl_stop, params_llw

In [7]:
def get_ptsz(avg_change_path): #Load list of patient and seizure names
    os.chdir(avg_change_path)
    
    df_params = open_xl('OPSCEAparams.xlsx') # Ndimensions and params_list

    with open('sz_name_list.csv','r') as sz_names:
        sz_names_list = list(csv.reader(sz_names))
        sz_rows = np.size(sz_names_list,0) #find row num 
        pt_data = []
        sz_data= []
        for r in range(0,sz_rows):  #collect symptom data for specfic seizure in list
            pt_data.append(sz_names_list[r][0]) 
            sz_data.append(sz_names_list[r][1]) 
    return pt_data, sz_data, df_params

## Input symptom, mode, and duration

In [8]:
sx_input = 'lhx'
mx_input = '2'
perdur_input = '10'

In [9]:
neuroanat_list = ['frontalpole', #FRONTAL LOBE
    'parstriangularis',
    'parsopercularis',
    'parsorbitalis',
    'rostralmiddlefrontal',
    'caudalmiddlefrontal',
    'lateralorbitofrontal',
    'superiorfrontal',
    'medialorbitofrontal',
    'precentral',
    'postcentral', # PARIETAL LOBE
    'inferiorparietal',   
    'superiorparietal',
    'supramarginal',
    'temporalpole', # TEMPORAL LOBE
    'middletemporal',
    'superiortemporal',
    'inferiortemporal',
    'parahippocampal',               
    'Right-Hippocampus',
    'Left-Hippocampus',
    'Right-Amygdala',
    'Left-Amygdala',
    'entorhinal',
    'bankssts',
    'fusiform', # OCCIPITAL LOBE                
    'lingual',
    'Right-Inf-Lat-Vent', # OTHER
    'Right-Cerebral-White-Matter',
    'Left-Cerebral-White-Matter',
    'Right-choroid-plexus',
    'Right-Putamen',
    'Right-VentralDC'];

## For loop through patients in list matching seizure

In [13]:
avg_change_path = '/Users/nataliasucher/Desktop/UCSF/Coding/OPSCEA/OPSCEADATA/avg_change_folders/'

out_ptsz = get_ptsz(avg_change_path)
pt_data = out_ptsz[0]
sz_data = out_ptsz[1]
df_params = out_ptsz[2]

# for neuro_struc in neuroanat_list:

for pt in pt_data: #For loop through patients 

    pt_i = pt_data.index(pt)
    sz_name = sz_data[pt_i]

    pt_path = avg_change_path + pt + '/'
    os.chdir(pt_path)
    pt_dir = os.listdir(pt_path)

    for sz_name in sz_data: 

        if sz_name in pt_dir: #patients in list matching seizure

            print(sz_name)
            out_elecs_anat = load_elecs_anat(pt_path)            
            out_params = get_params(df_params,pt)
            
            print(out_params)
            
            sz_path = pt_path + sz_data[pt_i]
            os.chdir(sz_path)

            #load semiology matrix .csv
            df = pd.read_csv(sz_name + '_mat.csv',usecols = [sx_input])
            
            #load frame speed and electrode matrix
            sz_mat = loadmat(sz_name +'.mat') # process seizures with symptom/mode     
            fs = sz_mat['fs'].flatten()
            # print(fs[0])
#             ppEEG = sz_mat['ppEEG']
#             print(np.shape(ppEEG))
#             # print(ppEEG[0][49459])
#             t_ppEEG = ppEEG.T
#             print(np.shape(t_ppEEG))
            

#             #remove EKG lines at end of anatomy + ppEEG?

#             try:
#                 ll = ll_weights(sz_name,perdur_input,out_elecs_anat[0],out_params[2],ppEEG,t_ppEEG,df,fs)
#                 # ll_onset = ll[0] #onset
#                 # t_ll_o = ll_onset.transpose()
#                 # print(ll_onset.T)
#                 # ll_before = ll[1] #before
#                 # ll_after = ll[2] #after
# #                 print(ll_onset[0])
# #                 print('----------------------------')
                
#                 b,a = signal.butter(2,range(1,(round(fs[0][0]/2-1)))/(fs[0][0]/2),'bandpass') # [b,a]=butter(2,[1 round(sfx/2-1)]/(sfx/2),'bandpass'); 
#                ll_onset[:]= signal.filtfilt(b,a,ll_onset)  # d(nns,:) = filtfilt(b,a,d(nns,:)')';                
# #                 # ll_before[:]= signal.filtfilt(b,a,ll_before.T)  # d(nns,:) = filtfilt(b,a,d(nns,:)')';
# #                 # ll_after[:]= signal.filtfilt(b,a,ll_after.T)  # d(nns,:) = filtfilt(b,a,d(nns,:)')';
# #                 t_onset = ll_onset.transpose()
# #                 print(t_onset[0])

#             except:
#                 print('no ll list')



EC96_01
(1, 9, 0.5)
EC107_01
(10, 20, 0.5)
EC133_03
(119, 129, 0.5)
EC229_02
(2, 16, 0.5)
